#快手活跃用户预测——lctry队解决方案
##简要介绍
###赛题：

“快手”新注册用户脱敏和采样后的数据30天，预测未来7天活跃的用户

###解决方案：滑窗法

主要使用lgb模型，xgb、catboost提升微小。另外使用了三个NN模型（keras+tensorflow）。
前两个NN结构相似，都是把mlp,lstm,cnn集合在一个网络中。

lgb线下0.8905~0.891，三个NN线下都可以0.891+

第一个NN训练方式非常对新手友好，

第二个NN训练比较正常。

第三个NN为GBDT特征（使用xgb提取）+deepFM，主要是对网上的开源代码做了点修改，以个人理解实现。

（NN新手，希望有老手提提意见）

###滑窗法对近期用户预测不准：

另外对26-30的用户使用单天滑窗，单独提取特征，使用5个lgb模型进行单独预测

完整的见Github: https://github.com/chantcalf/2018-Rank4-

#1 读取数据，处理异常

复赛异常数据好像和初赛异常数据不一样？（亏了一点点），树模型和NN融合后收益大幅度降低，因为NN对异常用户预测比较准

In [ ]:
#读取数据
preg = '/mnt/datasets/fusai/user_register_log.txt'
papp = '/mnt/datasets/fusai/app_launch_log.txt'
pv = '/mnt/datasets/fusai/video_create_log.txt'
pact = '/mnt/datasets/fusai/user_activity_log.txt'

regname = ['user_id','register_day','register_type','device_type']
actname = ['user_id','day','page','video_id','author_id','action_type']

regd = {'user_id':'uint32','register_day':'uint8',
        'register_type':'uint8','device_type':'uint32'} 
appd = {'user_id':'uint32','day':'uint8'}
actd = {'user_id':'uint32','day':'uint8',
        'page':'uint8','video_id':'uint32',
        'author_id':'uint32','action_type':'uint8'
        }

register = pd.read_csv(preg,sep="\t",names = regname,dtype=regd)
app = pd.read_csv(papp,names=['user_id','day'],sep='\t',dtype=appd)
video = pd.read_csv(pv,names=['user_id','day'],sep='\t',dtype=appd)
act = pd.read_csv(pact,names=actname,sep='\t',dtype=actd)

#异常用户
dropa = register[(register['register_day']==24) & (register['register_type']==3) & (register['device_type']==1)][['user_id']]
dropb = register[(register['register_day']==24) & (register['register_type']==3) &(register['device_type']==83)][['user_id']]
dropc = register[(register['register_day']==24) & (register['register_type']==3) &(register['device_type']==223)][['user_id']]
dropa = dropa.append(dropb).append(dropc)
dropa['probability'] = 0  
register = register[(register['register_day']!=24) | (register['register_type']!=3) | (register['device_type']!=1)]
register = register[(register['register_day']!=24) | (register['register_type']!=3) | (register['device_type']!=83)]
register = register[(register['register_day']!=24) | (register['register_type']!=3) | (register['device_type']!=223)]
del dropb,dropc
gc.collect()

#2 特征提取

##2.1 lgb特征

In [ ]:
def get_features_act(df,d2):
    df['register_time'] = d2-df.register_day+1
    tact = act[(act.day<=d2)]
    tapp = app[(app.day<=d2)]
    tvideo = video[(video.day<=d2)]
    tapp.day = d2 - tapp.day
    tact.day = d2 - tact.day
    tvideo.day = d2 - tvideo.day
    
    gp0 = tact.groupby(['user_id'])
    
    gp = gp0.size().rename('act').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['act'] = df['act'].astype('float32')
    
    gp = gp0['day'].nunique().rename('act_u').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')

    df['act_mean'] = (df['act']/df['act_u']).astype('float32')
    df['act_rate'] = df['act_u']/df['register_time']
    
    gp = gp0['day'].agg({
        'act_day_min':min,
        'act_day_max':max,
        'act_day_mean':np.mean,
        'act_day_std':np.std,
    }).reset_index()       
    df = df.merge(gp,on=['user_id'],how='left')
    df['act_day_gap'] = (df['act_day_max'] - df['act_day_min']).astype('float16')
    
    def get_last_gap(s):
        s = list(s)
        n = len(s)
        if n>1:
            s.sort()
            return s[1]-s[0]
        return None
        
    gp = gp0['day'].unique().apply(get_last_gap).rename('act_last_gap').reset_index()
    df = df.merge(gp,on=['user_id'],how='left') 
    
    gp = tapp[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_last_gap).rename('app_last_gap').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')  
    df['app_last_gap'] = df['app_last_gap'].astype('float16')
    
    gp = tvideo[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_last_gap).rename('video_last_gap').reset_index()
    df = df.merge(gp,on=['user_id'],how='left') 
    df['video_last_gap'] = df['video_last_gap'].astype('float16')

    for i in range(7):
        gp = tact[tact.day<=i][['user_id']].groupby(['user_id']).size().rename('last_'+str(i)+'_days_act').reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        df['last_'+str(i)+'_days_act'] = df['last_'+str(i)+'_days_act'].fillna(0).astype('uint16')

    for c in [0,2,3]: 
        gp = tact[(tact.day<16) & (tact['page']==c)][['user_id']].groupby(['user_id']).size().rename('act_page_'+str(c)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')

    gp = tact[(tact.day<16) ][['user_id','author_id']].groupby(['user_id'])['author_id'].nunique().rename('act_author_id_u').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['act_author_id_u'] = df['act_author_id_u'].fillna(0).astype('uint32')
    
    df['act_author_id_u_mean'] = (df['act_author_id_u']/df['register_time']).astype('float32')
    
    gp = tact[(tact.day<16) ][['user_id','video_id']].groupby(['user_id'])['video_id'].nunique().rename('act_video_id_u').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['act_video_id_u'] = df['act_video_id_u'].fillna(0).astype('uint32')
    
    df['act_video_id_u_mean'] = (df['act_video_id_u']/df['register_time']).astype('float32')

    df['author_id'] = df['user_id']
    gp = tact[(tact.day<7) & (tact['author_id']!=tact['user_id'])].groupby(['author_id']).size().rename('author').reset_index()
    df = df.merge(gp,on=['author_id'],how='left')

    gp = tact[(tact.day<7) & (tact['author_id']==tact['user_id'])][['user_id','author_id']].groupby(['user_id']).size().rename('author_self').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')

    gp = tact[(tact.day<7) & (tact.action_type==0)].groupby(['user_id','video_id']).size().rename('video_count').reset_index()
    gp1 = gp.groupby(['user_id'])['video_count'].max().rename('max_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[(tact.day<7) & (tact.action_type==0)].groupby(['user_id','author_id']).size().rename('author_count').reset_index()
    gp1 = gp.groupby(['user_id'])['author_count'].max().rename('max_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[(tact.day<7) & (tact.action_type==0)].groupby(['user_id','author_id'])['video_id'].nunique().rename('author_video_count').reset_index()
    gp1 = gp.groupby(['user_id'])['author_video_count'].max().rename('max_author_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[(tact.day<7) & (tact.page==0) & (tact.action_type==0)].groupby(['user_id'])['video_id'].nunique().rename('act00_video').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[(tact.day<7) & (tact.page==0) & (tact.action_type==0)].groupby(['user_id'])['author_id'].nunique().rename('act00_author').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')

    gp = tact[(tact.day<7) & (tact.page==2) & (tact.action_type==0)].groupby(['user_id'])['video_id'].nunique().rename('act20_video').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[(tact.day<7) & (tact.page==3) & (tact.action_type==0)].groupby(['user_id'])['author_id'].nunique().rename('act30_author').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[(tact.day<7) &(tact.page==2) & (tact.action_type==2)].groupby(['user_id'])['author_id'].nunique().rename('act22').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[(tact.day<7) &(tact.page==3) & (tact.action_type==2)].groupby(['user_id'])['author_id'].nunique().rename('act32').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    del df['author_id'],df['register_day'],df['register_type'],df['device_type']
    print (df.shape,'act ok!')
    return df

def get_features(df,d1,d2):
    df['register_time'] = d2-df.register_day+1
    tapp = app[(app.day>=d1) & (app.day<=d2)]
    tact = act[(act.day>=d1) & (act.day<=d2)]
    tvideo = video[(video.day>=d1) & (video.day<=d2)]
    tapp.day = d2 - tapp.day
    tact.day = d2 - tact.day
    tvideo.day = d2 - tvideo.day
    
    gp = tapp[['user_id']].groupby(['user_id']).size().rename('app').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['app'] = df['app'].astype('float32')
    
    df['app_mean'] = (df['app']/df['register_time']).astype('float32')
    
    for i in range(7):
        gp = tapp[tapp.day<=i][['user_id']].groupby(['user_id']).size().rename('last_'+str(i)+'_days_app').reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        df['last_'+str(i)+'_days_app'] = df['last_'+str(i)+'_days_app'].astype('float32')
    
    gp = tapp[['user_id','day']].groupby(['user_id'])['day'].agg({
        'app_day_min':min,
        'app_day_max':max,
        'app_day_mean':np.mean,
        'app_day_std':np.std,
    }).reset_index()    
    df = df.merge(gp,on=['user_id'],how='left')
    
    df['app_day_gap'] = (df['app_day_max'] - df['app_day_min']).astype('float16') 
    
    tapp = tapp.sort_values(by=['user_id','day'])
    tapp['diff'] = tapp.groupby('user_id')['day'].diff(-1).fillna(0)
    gp = tapp.groupby('user_id')['diff'].agg({
        #'app_diff_mean':np.mean,
        'app_diff_std':np.std,
        #'app_diff_max':np.max
    }).reset_index()
    df = pd.merge(df,gp,on=['user_id'],how='left',copy=False)
    
    gp = tvideo[['user_id']].groupby(['user_id']).size().rename('video').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['video'] = df['video'].astype('float32')
    
    df['video_mean'] = (df['video']/df['register_time']).astype('float32')

    gp = tvideo[['user_id','day']].groupby(['user_id'])['day'].agg({
        'video_day_min':min,
        'video_day_max':max,
        'video_day_mean':np.mean,
        'video_day_std':np.std,
    }).reset_index()       
    
    df = df.merge(gp,on=['user_id'],how='left')
    
    df['video_day_gap'] = (df['video_day_max'] - df['video_day_min']).astype('float16')
    
    gp = tvideo[['user_id','day']].groupby(['user_id'])['day'].nunique().rename('video_u').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')

    df['video_rate1'] = df['video_u']/df['app']
    
    def get_max_continue_days(s):
        s = list(s)
        s.sort()
        ans = 0
        t = 0
        for i in range(len(s)):
            if s[i]>0:
                t = t+1
            else:
                if t>ans:
                    ans = t
                t = 0
        if ans<t:
            ans = t
        return ans
        
    gp = tapp[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_max_continue_days).rename('max_continue_app').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[['user_id']].groupby(['user_id']).size().rename('16act').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')

    for i in [0,1,2]:
        gp = tact[tact.page==i][['user_id']].groupby(['user_id']).size().rename('16act_page'+str(i)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        df['16act_page'+str(i)] = df['16act_page'+str(i)]/df['16act']
        
    for i in [0,1,2]:
        gp = tact[tact.action_type==i][['user_id']].groupby(['user_id']).size().rename('16act_action'+str(i)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        df['16act_action'+str(i)] = df['16act_action'+str(i)]/df['16act']
        
    gp = tact[tact.action_type==0][['user_id','video_id']].groupby(['user_id','video_id']).size().rename('user_video_c').reset_index()
    gp1 = gp[gp['user_video_c']>1][['user_id']].groupby(['user_id']).size().rename('0love_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.action_type==3][['user_id','video_id']].groupby(['user_id','video_id']).size().rename('user_video_c').reset_index()
    gp1 = gp[gp['user_video_c']>1][['user_id']].groupby(['user_id']).size().rename('3love_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.action_type==0][['user_id','video_id','author_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('uav_u').reset_index()
    gp1 = gp[gp['uav_u']>1][['user_id']].groupby(['user_id']).size().rename('0love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.action_type==1][['user_id','video_id','author_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('uav_u').reset_index()
    gp1 = gp[gp['uav_u']>1][['user_id']].groupby(['user_id']).size().rename('1love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.action_type==3][['user_id','video_id','author_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('uav_u').reset_index()
    gp1 = gp[gp['uav_u']>1][['user_id']].groupby(['user_id']).size().rename('3love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.day<7][tact.action_type==0][['user_id','video_id']].groupby(['user_id','video_id']).size().rename('user_video_c').reset_index()
    gp1 = gp[gp['user_video_c']>1][['user_id']].groupby(['user_id']).size().rename('70love_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.day<7][tact.action_type==3][['user_id','video_id']].groupby(['user_id','video_id']).size().rename('user_video_c').reset_index()
    gp1 = gp[gp['user_video_c']>1][['user_id']].groupby(['user_id']).size().rename('73love_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.day<7][tact.action_type==0][['user_id','video_id','author_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('uav_u').reset_index()
    gp1 = gp[gp['uav_u']>1][['user_id']].groupby(['user_id']).size().rename('70love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.day<7][tact.action_type==1][['user_id','video_id','author_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('uav_u').reset_index()
    gp1 = gp[gp['uav_u']>1][['user_id']].groupby(['user_id']).size().rename('71love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.day<7][tact.action_type==3][['user_id','video_id','author_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('uav_u').reset_index()
    gp1 = gp[gp['uav_u']>1][['user_id']].groupby(['user_id']).size().rename('73love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.action_type==0][['user_id','video_id', 'day']].groupby(['user_id','video_id'])['day'].nunique().rename('user_video_d').reset_index()
    gp1 = gp[gp['user_video_d']>1][['user_id']].groupby(['user_id']).size().rename('d0love_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.action_type==0][['user_id','author_id', 'day']].groupby(['user_id','author_id'])['day'].nunique().rename('user_author_d').reset_index()
    gp1 = gp[gp['user_author_d']>1][['user_id']].groupby(['user_id']).size().rename('d0love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.day<7][tact.action_type==0][['user_id','video_id', 'day']].groupby(['user_id','video_id'])['day'].nunique().rename('user_video_d').reset_index()
    gp1 = gp[gp['user_video_d']>1][['user_id']].groupby(['user_id']).size().rename('7d0love_video_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[tact.day<7][tact.action_type==0][['user_id','author_id', 'day']].groupby(['user_id','author_id'])['day'].nunique().rename('user_author_d').reset_index()
    gp1 = gp[gp['user_author_d']>1][['user_id']].groupby(['user_id']).size().rename('7d0love_author_count').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    del df['register_time'],df['register_day'],df['register_type'],df['device_type']
    print (df.shape,'ok')
    return df

#去除注册天再提取特征
def get_features1(df,d1,d2):
    tapp = app[(app.day<=d2)].copy()
    tact = act[(act.day<=d2)].copy()
    tvideo = video[(video.day<=d2)].copy()
    
    tapp = tapp.merge(register[['user_id','register_day']],on=['user_id'],how='left')
    tvideo = tvideo.merge(register[['user_id','register_day']],on=['user_id'],how='left')
    tact = tact.merge(register[['user_id','register_day']],on=['user_id'],how='left')
    
    tapp = tapp[tapp.day>tapp.register_day]
    tvideo = tvideo[tvideo.day>tvideo.register_day]
    tact = tact[tact.day>tact.register_day]
    
    tapp.day = d2 - tapp.day
    tact.day = d2 - tact.day
    tvideo.day = d2 - tvideo.day
    
    def getrt(x):
        if x<16:
            return x
        return 16
        
    df['rt'] = df['register_time'].apply(getrt)
    
    gp = tapp[tapp.day<16].groupby(['user_id']).size().rename('no_app').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['no_app_r'] = df['no_app']/df['rt']
    
    temp = tact[tact.day<16].groupby(['user_id'])
    gp = temp.size().rename('no_act').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = temp['video_id'].nunique().rename('no_act_video').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    gp = temp['author_id'].nunique().rename('no_act_author').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    df['no_act_video_r'] = df['no_act_video']/df['rt']
    df['no_act_video_r1'] = df['no_act_video']/df['no_app']
    df['no_act_author_r'] = df['no_act_author']/df['rt']
    df['no_act_author_r1'] = df['no_act_author']/df['no_app']
    
    del temp
    gc.collect()
    
    del df['rt'],df['no_act_video'],df['no_act_author']
    print (df.shape,'ok')
    return df 

#注册特征
def get_reg_features():
    df = register[['user_id','register_type','device_type']]
    tact0 = act.merge(register[['user_id','register_day']],on=['user_id'],how='left')
    tvideo0 = video.merge(register[['user_id','register_day']],on=['user_id'],how='left')
    tapp0 = app.merge(register[['user_id','register_day']],on=['user_id'],how='left')
    
    tact = tact0[tact0.day==tact0.register_day]
    gp = tact.groupby(['user_id']).size().rename('regday_act').reset_index()
    avg_regday_act = gp['regday_act'].mean()
    def sp_regday_act(x):
        if x>=avg_regday_act:
            return 2
        return 1
    gp['regday_act'] = gp['regday_act'].apply(sp_regday_act)
    df = df.merge(gp,on=['user_id'],how='left')
    df['regday_act'] = df['regday_act'].fillna(0)
    
    gp = tact[tact['action_type']==2].groupby(['user_id']).size().rename('regday_action2').reset_index()
    gp['regday_action2'] = 1
    df = df.merge(gp,on=['user_id'],how='left')
    df['regday_action2'] = df['regday_action2'].fillna(0)
    
    for i in [0,1,3]:
        gp = tact[tact['action_type']==i].groupby(['user_id']).size().rename('regday_action'+str(i)).reset_index()
        gp['regday_action'+str(i)] = 1
        df = df.merge(gp,on=['user_id'],how='left')
        df['regday_action'+str(i)] = df['regday_action'+str(i)].fillna(0)
    
    for i in [1,2,3]:
        gp = tact[tact['page']==i].groupby(['user_id']).size().rename('regday_page'+str(i)).reset_index()
        gp['regday_page'+str(i)] = 1
        df = df.merge(gp,on=['user_id'],how='left')
        df['regday_page'+str(i)] = df['regday_page'+str(i)].fillna(0)
        
    tvideo = tvideo0[tvideo0.day==tvideo0.register_day]
    gp = tvideo.groupby(['user_id']).size().rename('regday_video').reset_index()
    avg_regday_video = gp['regday_video'].mean()
    def sp_regday_video(x):
        if x>=avg_regday_video:
            return 2
        return 1
    gp['regday_video'] = gp['regday_video'].apply(sp_regday_video)
    df = df.merge(gp,on=['user_id'],how='left')
    df['regday_video'] = df['regday_video'].fillna(0)
    
    print (df.shape,'ok')
    return df    

#train和test的共同特征
def get_features_all(df,df1):
    print (df.shape)
    lendf = len(df)
    
    df= df.append(df1).reset_index(drop=True)
    del df1
    gc.collect()
    
    gp = register[['device_type']].groupby(['device_type']).size().rename('device_type_count').reset_index()
    gp = gp.sort_values(by='device_type_count',ascending=False)
    r = list(range(len(gp)))
    gp['r'] = r
    
    def rankfen(x):
        a = [5000,1000,500,0]
        for i in range(len(a)):
            if x>=a[i]:
                return i
        return -1
    gp['r'] = gp['r'].apply(rankfen)
    
    df = df.merge(gp,on=['device_type'],how='left')
    
    device_count = list(gp['device_type_count'])
    device_count.sort(reverse = True)
    print (len(device_count))
    print (device_count[10],device_count[50],device_count[100],device_count[500])
    
    df.loc[df['device_type_count']<device_count[500],'device_type_count'] = 0
    df.loc[(df['device_type_count']>0) & (df['device_type_count']<device_count[100]),'device_type_count'] = 1
    df.loc[(df['device_type_count']>0) & (df['device_type_count']<device_count[50]),'device_type_count'] = 2

    gp = register[['device_type','register_type']].groupby(['register_type','device_type']).size().rename('device_register_count').reset_index()
    df = df.merge(gp,on=['register_type','device_type'],how='left')
    gp = register[['register_type']].groupby(['register_type']).size().rename('register_count').reset_index()
    df = df.merge(gp,on=['register_type'],how='left')
    df['device_register_count/register_count'] = df['device_register_count']/df['register_count']

    del df['user_id'],df['register_count']
    df1 = df[lendf:].reset_index(drop=True)
    df = df[:lendf]
    print (df.shape)
    return df,df1

##2.2 26-30特征

In [ ]:
#预测注册k天后七天是否活跃，单天滑窗提取特征    
def get_features27(k):
    df = register[register.register_day<=23-k].reset_index(drop=True)
    test = register[register.register_day==30-k].reset_index(drop=True)
    
    app0 = pd.merge(app,register[['user_id','register_day']],on=['user_id'],how='left')
    video0 = pd.merge(video,register[['user_id','register_day']],on=['user_id'],how='left')
    act0 = pd.merge(act,register[['user_id','register_day']],on=['user_id'],how='left')
    
    app0 = app0[(app0.register_day<=23-k) | (app0.register_day==30-k)]
    video0 = video0[(video0.register_day<=23-k) | (video0.register_day==30-k)]
    act0 = act0[(act0.register_day<=23-k) | (act0.register_day==30-k)]
    
    def active30(df):
        tapp = app0[(app0['day']>app0['register_day']+k) & (app0['day']<=app0['register_day']+k+7)]
        tvideo = video0[(video0['day']>video0['register_day']+k) & (video0['day']<=video0['register_day']+k+7)]   
        tact = act0[(act0['day']>act0['register_day']+k) & (act0['day']<=act0['register_day']+k+7)]
                   
        gp1 = tapp[['user_id']].groupby(['user_id']).size().rename('app').reset_index()
        gp2 = tvideo[['user_id']].groupby(['user_id']).size().rename('video').reset_index()
        gp3 = tact[['user_id']].groupby(['user_id']).size().rename('act').reset_index()
           
        df = df.merge(gp1,on=['user_id'],how='left')
        df = df.merge(gp2,on=['user_id'],how='left')
        df = df.merge(gp3,on=['user_id'],how='left')
        
        df['app'] = df['app'].fillna(0)
        df['video'] = df['video'].fillna(0)
        df['act'] = df['act'].fillna(0)
        df['count'] = df['app']+df['video']+df['act']
        
        def isnotzero(x):
            if x>0:
                return 1
            return 0
        y = df['count'].apply(isnotzero)      
        return y

    yy = active30(df)
    df['Y'] = yy
    test['Y'] = -1
    df = df.append(test).reset_index(drop=True)
    df['author_id'] = df['user_id']
    
    tvideo = video0[video0['day']==video0['register_day']]
    tact = act0[act0['day']==act0['register_day']]               
    '''
    gp = tvideo.groupby(['user_id']).size().rename('video').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
      
    gp = tvideo.groupby(['user_id','register_day']).size().rename('videodaycount').reset_index()
    gp1 = gp.groupby(['register_day'])['videodaycount'].mean().rename('videodaymean').reset_index()
    df = df.merge(gp1,on=['register_day'],how='left')
    df['videorate'] = (df['video']/df['videodaymean']).astype('float32')
    del df['videodaymean'],df['video']
    '''
    gp = tact[['user_id']].groupby(['user_id']).size().rename('act').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
      
    gp = tact[['user_id','register_day']].groupby(['user_id','register_day']).size().rename('actdaycount').reset_index()
    gp1 = gp.groupby(['register_day'])['actdaycount'].mean().rename('actdaymean').reset_index()
    df = df.merge(gp1,on=['register_day'],how='left')
    
    df['actrate'] = (df['act']/df['actdaymean']).astype('float32')
    del df['actdaymean']
   
    gp = tact[['user_id','author_id']].groupby(['user_id'])['author_id'].nunique().rename('author#').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[['user_id','author_id','video_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('video_count').reset_index()
    gp1 = gp.groupby(['user_id'])['video_count'].max().rename('video_count_max').reset_index()
    df = df.merge(gp1,on=['user_id'],how='left')
    
    gp = tact[['user_id','video_id']].groupby(['user_id'])['video_id'].nunique().rename('video#').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    for c in [1,2,3]:
        gp = tact[tact['page']==c][['user_id']].groupby(['user_id']).size().rename('act_page'+str(c)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        df['act_pagerate'+str(c)] = (df['act_page'+str(c)]/df['act']).astype('float32')
    
    for c in [1,2,3]:
        gp = tact[tact['action_type']==c][['user_id']].groupby(['user_id']).size().rename('act_action'+str(c)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        df['act_actionrate'+str(c)] = (df['act_action'+str(c)]/df['act']).astype('float32')
    
    gp = tact[tact['author_id']!=tact['user_id']][tact.action_type==0][['author_id']].groupby(['author_id']).size().rename('be_author0').reset_index()
    df = df.merge(gp,on=['author_id'],how='left')
    
    #gp = tact[tact['author_id']!=tact['user_id']][tact.action_type==1][['author_id']].groupby(['author_id']).size().rename('be_author1').reset_index()
    #df = df.merge(gp,on=['author_id'],how='left')
    
    gp = tact[tact['author_id']==tact['user_id']][tact.action_type==0][['author_id']].groupby(['author_id']).size().rename('self_author0').reset_index()
    df = df.merge(gp,on=['author_id'],how='left')
    
    #gp = tact[tact['author_id']==tact['user_id']][tact.action_type==3][['author_id']].groupby(['author_id']).size().rename('self_author3').reset_index()
    #df = df.merge(gp,on=['author_id'],how='left')
    
    for i in range(1,k+1):
        tapp = app0[app0['day']==app0['register_day']+i]
        tvideo = video0[video0['day']==video0['register_day']+i]
        tact = act0[act0['day']==act0['register_day']+i]               
        gp = tapp.groupby(['user_id']).size().rename(str(i)+'app').reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        
        gp = tvideo.groupby(['user_id']).size().rename(str(i)+'video').reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        
        gp = tact[['user_id']].groupby(['user_id']).size().rename(str(i)+'act').reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
          
        gp = tact[['user_id','register_day']].groupby(['user_id','register_day']).size().rename('actdaycount').reset_index()
        gp1 = gp.groupby(['register_day'])['actdaycount'].mean().rename('actdaymean').reset_index()
        df = df.merge(gp1,on=['register_day'],how='left')
        
        df[str(i)+'actrate'] = (df[str(i)+'act']/df['actdaymean']).astype('float32')
        del df['actdaymean']
       
        gp = tact[['user_id','author_id']].groupby(['user_id'])['author_id'].nunique().rename(str(i)+'author#').reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        
        gp = tact[['user_id','author_id','video_id']].groupby(['user_id','author_id'])['video_id'].nunique().rename('video_count').reset_index()
        gp1 = gp.groupby(['user_id'])['video_count'].max().rename(str(i)+'video_count_max').reset_index()
        df = df.merge(gp1,on=['user_id'],how='left')
        
        gp = tact[['user_id','video_id']].groupby(['user_id'])['video_id'].nunique().rename(str(i)+'video#').reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        
        for c in [1,2,3]:
            gp = tact[tact['page']==c][['user_id']].groupby(['user_id']).size().rename(str(i)+'act_page'+str(c)).reset_index()
            df = df.merge(gp,on=['user_id'],how='left')
            #df[str(i)+'act_pagerate'+str(c)] = (df[str(i)+'act_page'+str(c)]/df[str(i)+'act']).astype('float32')
        
        for c in [1,2,3]:
            gp = tact[tact['action_type']==c][['user_id']].groupby(['user_id']).size().rename(str(i)+'act_action'+str(c)).reset_index()
            df = df.merge(gp,on=['user_id'],how='left')
            #df[str(i)+'act_actionrate'+str(c)] = (df[str(i)+'act_action'+str(c)]/df[str(i)+'act']).astype('float32')
        
        gp = tact[tact['author_id']!=tact['user_id']][tact.action_type==0][['author_id']].groupby(['author_id']).size().rename(str(i)+'be_author0').reset_index()
        df = df.merge(gp,on=['author_id'],how='left')
        
        gp = tact[tact['author_id']==tact['user_id']][tact.action_type==0][['author_id']].groupby(['author_id']).size().rename(str(i)+'self_author0').reset_index()
        df = df.merge(gp,on=['author_id'],how='left')
    
    tapp = app0[(app0['day']>app0['register_day'])&(app0['day']<app0['register_day']+4)] 
    tvideo = video0[(video0['day']>video0['register_day'])&(video0['day']<video0['register_day']+4)]
    tact = act0[(act0['day']>act0['register_day'])&(act0['day']<act0['register_day']+4)] 
    #tapp.day = tapp.day-tapp.register_day
    #tvideo.day = tvideo.day-tvideo.register_day
    #tact.day = tact.day-tact.register_day
    
    gp = tapp.groupby(['user_id']).size().rename('kapp').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[['user_id']].groupby(['user_id']).size().rename('kact').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[['user_id','author_id']].groupby(['user_id'])['author_id'].nunique().rename('kauthor#').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tact[['user_id','video_id']].groupby(['user_id'])['video_id'].nunique().rename('kvideo#').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    del df['author_id']
    return df    

##2.3 NN特征

In [ ]:
#NN普通特征(使用时再拼上注册特征和train和test共有特征)
def get_features_nn(df,d2):
    df['register_time'] = d2-df.register_day+1
    tact = act[(act.day<=d2)]
    tapp = app[(app.day<=d2)]
    tvideo = video[(video.day<=d2)]
    tapp.day = d2 - tapp.day
    tact.day = d2 - tact.day
    tvideo.day = d2 - tvideo.day
                
    gp = tapp[['user_id']].groupby(['user_id']).size().rename('app').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['app'] = df['app'].fillna(0).astype('uint8')
    df['app_mean'] = (df['app']/df['register_time']).astype('float32')
    gp = tapp[['user_id','day']].groupby(['user_id'])['day'].agg({
        'app_day_min':min,
        'app_day_max':max,
        'app_day_mean':np.mean,
        'app_day_std':np.std,
    }).reset_index()    
    df = df.merge(gp,on=['user_id'],how='left')

    gp = tvideo[['user_id','day']].groupby(['user_id'])['day'].agg({
        'video_day_min':min,
        'video_day_max':max,
        'video_day_mean':np.mean,
        'video_day_std':np.std,
    }).reset_index()       
    
    df = df.merge(gp,on=['user_id'],how='left')
    
    gp = tvideo[['user_id']].groupby(['user_id']).size().rename('video').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['video'] = df['video'].fillna(0).astype('uint16')
    
    df['video_mean'] = (df['video']/df['register_time']).astype('float32')
    
    gp = tvideo[['user_id','day']].groupby(['user_id'])['day'].nunique().rename('video_u').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')

    df['video_rate1'] = df['video_u']/df['app']
    
    gp0 = tact.groupby(['user_id'])
    gp = gp0.size().rename('act').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['act'] = df['act'].fillna(0).astype('uint32')
    df['act_mean'] = (df['act']/df['register_time']).astype('float32')
    
    gp = gp0['day'].nunique().rename('act_u').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    df['act_rate'] = df['act_u']/df['register_time']
    gp = gp0['day'].agg({
        'act_day_min':min,
        'act_day_max':max,
        'act_day_mean':np.mean,
        'act_day_std':np.std,
    }).reset_index()       
    df = df.merge(gp,on=['user_id'],how='left')
    
    def get_last_gap(s):
        s = list(s)
        n = len(s)
        if n>1:
            s.sort()
            return s[1]-s[0]
        return None
        
    gp = gp0['day'].unique().apply(get_last_gap).rename('act_last_gap').reset_index()
    df = df.merge(gp,on=['user_id'],how='left') 
    
    gp = tapp[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_last_gap).rename('app_last_gap').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')  
    
    gp = tvideo[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_last_gap).rename('video_last_gap').reset_index()
    df = df.merge(gp,on=['user_id'],how='left')
    
    del df['register_day']
    print('nn features ok!',df.shape)
    return df
    
#NN时序特征
def get_features_nn1(d2):
    df = register[register.register_day<=d2][['user_id']]
    tact = act[(act.day<=d2)]
    tapp = app[(app.day<=d2)]
    tvideo = video[(video.day<=d2)]
    tapp.day = d2 - tapp.day
    tact.day = d2 - tact.day
    tvideo.day = d2 - tvideo.day
    
    for i in range(14):
        t1 = tact[(tact.day==i)]
        
        gp = tapp[tapp.day==i][['user_id']].groupby(['user_id']).size().rename('app'+str(i)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')
        df['app'+str(i)] = df['app'+str(i)].fillna(0).astype('uint8')
        
        gp = t1[['user_id']].groupby(['user_id']).size().rename('act'+str(i)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')    
        df['act'+str(i)] = df['act'+str(i)].fillna(0).astype('uint16')
        
        gp = tvideo[tvideo.day==i][['user_id']].groupby(['user_id']).size().rename('video'+str(i)).reset_index()
        df = df.merge(gp,on=['user_id'],how='left')   
        df['video'+str(i)] = df['video'+str(i)].fillna(0).astype('uint16')
        
        for j in [0,1,2,3]:
            gp = t1[t1.page==j][['user_id']].groupby(['user_id']).size().rename(str(i)+'act_page'+str(j)).reset_index()
            df = df.merge(gp,on=['user_id'],how='left')
            df[str(i)+'act_pager'+str(j)] = (df[str(i)+'act_page'+str(j)]/(df['act'+str(i)]+0.00001)).astype('float32')
            
        for j in [0,1,2,3]:
            gp = t1[t1.action_type==j][['user_id']].groupby(['user_id']).size().rename(str(i)+'act_action'+str(j)).reset_index()
            df = df.merge(gp,on=['user_id'],how='left')
            df[str(i)+'act_actionr'+str(j)] = (df[str(i)+'act_action'+str(j)]/(df['act'+str(i)]+0.00001)).astype('float32')    
        
        for j in [0,1,2,3]:
            t2 = t1[t1.page==j]
            for k in [0,1,2,3]:
                if j==1 and k==2:
                    continue
                gp =  t2[t2.action_type==k][['user_id']].groupby(['user_id']).size().rename('act'+str(i)+str(j)+str(k)).reset_index()
                df = df.merge(gp,on=['user_id'],how='left')
                df['act'+str(i)+str(j)+str(k)] = df['act'+str(i)+str(j)+str(k)].fillna(0).astype('uint16')
                
    print('nn features ok!',df.shape)
    return df   
    

##2.4 GBDT特征
使用xgb生成100（树的数量）维类别特征，因为树深为4，最多16个叶子，但取值不是0-15，使用前需LabelEncoder

In [ ]:
def gbdt_lr_predict_data(train_df,train_y,test_df):
    model = xgb.XGBClassifier(
        nthread=8,# cpu 线程数 默认最大
        n_estimators=100,
        max_depth=4,
        seed=5,
        learning_rate=0.07,
        subsample=0.9,
        min_child_weight=1,
        colsample_bytree=.9,
        gamma = 0.1,
        reg_alpha=0.7,
        reg_lambda=0.6,
        silent=False,
        eval_metric='auc'
        )
    model.fit(train_df,train_y)
    grd_res = model.predict(test_df) 
    #grd_enc = OneHotEncoder()
    train_data = model.apply(train_df)
    print (train_data.shape)
    test_data = model.apply(test_df)
    print (test_data.shape)
    return train_data,test_data

#3 树模型
普通的lgb，xgb，catboost，具体代码见github


#4 NN

#4.1 NN0
双输入模型，普通特征+时序特征

时序特征按时间顺序提取的，手动拉成二维

MLP使用 init='he_normal'，和PRelu效果比默认和Relu好些

BN和dropout是标配，BN正常在激活函数前面，但也可以放在激活函数之后，有时有提升

In [ ]:
def simple_keras0(train_df,dfa,train_y,test_df,dfb,val,y2):
    train_df =train_df.fillna(-1)#普通特征
    test_df = test_df.fillna(-1)
    dfa =dfa.fillna(0)#时序特征
    dfb = dfb.fillna(0)

    sc = StandardScaler()
    sc.fit(train_df)
    X = sc.transform(train_df)
    test_df = sc.transform(test_df)
    y = train_y.values
    
    sc.fit(dfa)
    X1 = sc.transform(dfa.values)
    test_df1 = sc.transform(dfb.values)
    
    def getlstmdata(s,s0):
        n = s.shape[1]
        m = n//14
        data = []
        for i in range(13,-1,-1):
            idx = list(range(m*(i),m*(i+1)))
            #temp = np.append(s[:,idx],s0,axis=1)
            data.append(s[:,idx])
   
        data = np.array(data)
        data = np.swapaxes(data,0,1)
        #data = np.swapaxes(data,1,2)
        print (data.shape)
        return data
    
    X1 = getlstmdata(X1,X)
    test_df1 = getlstmdata(test_df1,test_df)
        
    def get_model1():
        xin = Input(shape=(X.shape[1],))
        xin1 = Input(shape=(X1.shape[1],X1.shape[2]))
        
        x1 = Dense(32,init='he_normal')(xin)

        x1 = PReLU()(x1)
        x1b = x1
        x1 = BatchNormalization()(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Dense(16,init='he_normal')(x1)
        x1 = PReLU()(x1)
        x1 = BatchNormalization()(x1)
        x1 = Dropout(0.25)(x1)

        x1 = Dense(1)(x1)
 
        #x2 = Dense(64)(xin1)
        #x2 = PReLU()(x2)
        x2 = LSTM(36,activation='relu',dropout=0.2,return_sequences=True)(xin1)#多对多
        #x2 = LSTM(16,activation='relu',dropout=0.2,return_sequences=True)(x2)
        x2 = LSTM(18,activation='relu',dropout=0.2,return_sequences=False)(x2)#多对一
        x2b = x2
        #x2 = PReLU()(x2)
        x2 = BatchNormalization()(x2)
        x2 = Dropout(0.5)(x2)
        x2 = Dense(1)(x2)
        
        #可以堆多个一维卷积，但是效果不好，这里没加
        #可以用for循环，计算多种滤波尺寸的CNN，最后再拼接
        x3 = Conv1D(20,3)(xin1)#可以加same
        x3 = BatchNormalization()(x3)
        x3 = Activation('relu')(x3)
        x3 = MaxPooling1D(2)(x3)
        x3 = LSTM(10,activation='relu',dropout=0.2,return_sequences=False)(x3)
        #x3 = Conv1D(64,3)(x3)
        #x3 = BatchNormalization()(x3)
        #x3 = Activation('relu')(x3)
        #x3 = Lambda(lambda x: K.max(x,axis = 1))(x3)
        x3 = Dropout(0.5)(x3)
        x3 = Dense(1)(x3)
    
        xb = Concatenate(axis=1)([x1b,x2b])
        xb = Dense(16,init='he_normal')(xb)
        xb = PReLU()(xb)
        xb = BatchNormalization()(xb)
        xb = Dropout(0.25)(xb)
        
        xb = Dense(1)(xb)
        
        xx = Concatenate(axis=1)([x1,x2,xb,x3])
        y = Dense(1, activation='sigmoid')(xx)
        
        model = Model(inputs=[xin,xin1],outputs=y)
        return model

    from sklearn.model_selection import KFold
    N = 5
    skf = KFold(n_splits=N,shuffle=False,random_state=None)
    
    xx_cv = []
    xx_pre = []
    ii = 0
    
    for train_in,test_in in skf.split(X,y):
        X_train,X_test,y_train,y_test = X[train_in],X[test_in],y[train_in],y[test_in]
        X_train1 = X1[train_in]
        X_test1 = X1[test_in]
        
        model = get_model1()
        model.compile(loss='binary_crossentropy',
                      #optimizer=SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False),
                      optimizer = 'nadam',
                      #metrics=['accuracy']
                      )
        if ii==0:            
            print (model.summary())
        
        epochs = 3
        filepath = 'best_param'+str(ii)+'.h5'
        bests = 0
        beste = 0
        allres = []
        allval = []

        #for循环观察每轮结果，判断是否保存参数，是否更改学习率等等，手动挡，很灵活
        for i in range(20):
            print (i)
            model.fit([X_train,X_train1],y_train,epochs=1,batch_size=512,
                    validation_data=[[X_test,X_test1],y_test],
                    shuffle=True,verbose=2,)
            res = model.predict([X_test,X_test1],batch_size=1024)
            s = metrics.roc_auc_score(y_test, res)
            
            res1 = model.predict([test_df,test_df1],batch_size=1024)
            allres.append(res1)
            allval.append(s)
            print (s)
            if val:
                print(metrics.roc_auc_score(y2, res1))
            if s>bests:
                bests = s
                model.save(filepath)
                beste = i
            if i-beste>5:
                break       
        model.load_weights(filepath)    
        model.compile(loss='binary_crossentropy',
                      optimizer=SGD(lr=0.0004, momentum=0.9, nesterov=False),
                      #optimizer = 'nadam',
                      #metrics=['accuracy']
                      )
        for i in range(5):
            print (i)
            model.fit([X_train,X_train1],y_train,epochs=1,batch_size=512,
                    validation_data=[[X_test,X_test1],y_test],
                    shuffle=True,verbose=2,)
            res = model.predict([X_test,X_test1],batch_size=1000)
            s = metrics.roc_auc_score(y_test, res)
            
            res1 = model.predict([test_df,test_df1],batch_size=1000)
            allres.append(res1)
            allval.append(s)
            print (s)
            if val:
                print(metrics.roc_auc_score(y2, res1))
            if s>bests:
                bests = s
                model.save(filepath)
                beste = i
            if i-beste>10:
                break       
        #model.save(filepath)    
        model.load_weights(filepath)
        res1 = model.predict([test_df,test_df1],batch_size=1000)
        if val:
            s1 = metrics.roc_auc_score(y2, res1)
            print (s1)              
        xx_pre.append(res1)  
        xx_cv.append(bests)
        ii = ii + 1
        #if val:
        #break
    
    if val:
        test_y2 = xx_pre[0]
    else:
        test_y2 = 0
        for i in xx_pre:
            test_y2 = test_y2+i
        test_y2 = test_y2/N
    
    if val:
        print (metrics.roc_auc_score(y2, test_y2))

    return test_y2

#4.2 NN1

In [ ]:
#划分比较粗糙,这个auc会比实际值小
def auc(y_true, y_pred):  
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)  
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)  
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)  
    binSizes = -(pfas[1:]-pfas[:-1])  
    s = ptas*binSizes  
    return K.sum(s, axis=0)  

# PFA, prob false alert for binary classifier  
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # N = total number of negative labels  
    N = K.sum(1 - y_true)  
    # FP = total number of false alerts, alerts from the negative class labels  
    FP = K.sum(y_pred - y_pred * y_true)  
    return FP/N  

# P_TA prob true alerts for binary classifier  
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # P = total number of positive labels  
    P = K.sum(y_true)  
    # TP = total number of correct alerts, alerts from the positive class labels  
    TP = K.sum(y_pred * y_true)  
    return TP/P 

#train
def simple_keras(train_df,dfa,train_y,test_df,dfb,val,y2,numk):
    train_df =train_df.fillna(-1)
    test_df = test_df.fillna(-1)
    dfa =dfa.fillna(0)
    dfb = dfb.fillna(0)

    sc = StandardScaler()
    sc.fit(train_df)
    X = sc.transform(train_df)
    test_df = sc.transform(test_df)
    y = train_y.values
    
    sc.fit(dfa)
    X1 = sc.transform(dfa.values)
    test_df1 = sc.transform(dfb.values)
    
    def getlstmdata(s,s0):
        n = s.shape[1]
        m = n//14
        data = []
        for i in range(13,-1,-1):
            # = list(range(m*(i+1)-1,m*(i)-1,-1))
            idx = list(range(m*(i),m*(i+1)))
            #temp = np.append(s[:,idx],s0,axis=1)
            data.append(s[:,idx])
   
        data = np.array(data)
        data = np.swapaxes(data,0,1)
        #data = np.swapaxes(data,1,2)
        print (data.shape)
        return data
    
    X1 = getlstmdata(X1,X)
    test_df1 = getlstmdata(test_df1,test_df)
    
    
    def get_model_nn1():
        xin = Input(shape=(X.shape[1],))
        xin1 = Input(shape=(X1.shape[1],X1.shape[2]))
        
        x1 = Dense(32,init='he_normal')(xin)
        x1 = PReLU()(x1)
        x1 = BatchNormalization()(x1)
        x1b = x1

        x2 = LSTM(36,activation='relu',dropout=0.2,return_sequences=False)(xin1)
        x2 = BatchNormalization()(x2)
        x2b = x2

        x3 = Conv1D(20,3)(xin1)
        x3 = BatchNormalization()(x3)
        x3 = Activation('relu')(x3)
        x3 = MaxPooling1D(2)(x3)
        x3 = LSTM(10,activation='relu',dropout=0.2,return_sequences=False)(x3)
        x3 = BatchNormalization()(x3)
        x3b = x3
        
        xb = Concatenate(axis=1)([x1b,x2b,x3b])
        xb = Dense(64,init='he_normal')(xb)
        xb = PReLU()(xb)
        xb = BatchNormalization()(xb)
        xb = Dropout(0.25)(xb)
        xb = Dense(16,init='he_normal')(xb)
        xb = Dropout(0.5)(xb)
        y = Dense(1, activation='sigmoid')(xb)
        
        model = Model(inputs=[xin,xin1],outputs=y)
        return model

    #十折平均效果较差，可以使用随机数，每次随机生成随机种子，从中随机取一种划分方式，跑多个NN再平均
    from sklearn.model_selection import KFold
    N = 10
    skf = KFold(n_splits=N,shuffle=True,random_state=888)
    xx_cv = []
    xx_pre = []
    ii = 0
    for train_in,test_in in skf.split(X,y):
        if ii<9:
            ii = ii+1
            continue
        X_train,X_test,y_train,y_test = X[train_in],X[test_in],y[train_in],y[test_in]
        X_train1 = X1[train_in]
        X_test1 = X1[test_in]
        
        model = get_model_nn1()
        model.compile(loss='binary_crossentropy',
                      #optimizer=SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False),
                      optimizer = 'nadam',
                      #optimizer = 'nadam',
                      metrics=[auc]
                      )
        #if ii==0:            
            #print (model.summary())
        filepath = 'model_nn1_param'+str(numk)+'.h5'
        bests = 0
        beste = 0
        allres = []
        allval = []
        checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
        
        model.fit([X_train,X_train1],y_train,epochs=20,batch_size=1024,
                    validation_data=[[X_test,X_test1],y_test],
                    #validation_data=[[test_df,test_df1],y2],
                    shuffle=True,verbose=2,
                    callbacks = [EarlyStopping(monitor='val_auc',patience=5,mode='max'),checkpoint],
                    )
        
        model.load_weights(filepath)
        res = model.predict([X_test,X_test1],batch_size=4096)
        s = metrics.roc_auc_score(y_test, res)
        bests = s
        print (s)
        if val:
            res1 = model.predict([test_df,test_df1],batch_size=4096)
            print(metrics.roc_auc_score(y2, res1))
            
        model.compile(loss='binary_crossentropy',
                      optimizer=SGD(lr=0.0001, momentum=0.9,decay=0, nesterov=False),
                      metrics=[auc]
                      )
        for i in range(10):
            print (i)
            model.fit([X_train,X_train1],y_train,epochs=1,batch_size=1024,
                    validation_data=[[X_test,X_test1],y_test],
                    shuffle=True,verbose=2,)
            res = model.predict([X_test,X_test1],batch_size=1024)
            s = metrics.roc_auc_score(y_test, res)
            
            res1 = model.predict([test_df,test_df1],batch_size=4096)
            allres.append(res1)
            allval.append(s)
            print (s)
            if val:
                print(metrics.roc_auc_score(y2, res1))
            if s>bests:
                bests = s
                model.save(filepath)
                beste = i
            if i-beste>=2:
                break   
               
        #model.save(filepath)    
        model.load_weights(filepath)
        res1 = model.predict([test_df,test_df1],batch_size=4096)
        if val:
            s1 = metrics.roc_auc_score(y2, res1)
            print (s1)              
        xx_pre.append(res1)  
        xx_cv.append(bests)
        ii = ii + 1
        #if val:
        break
    
    test_y2 = xx_pre[0]
    return test_y2

#4.3 GBDT+deepFM
输入为100维类别特征
deepFM就是一阶+二阶+MLP
可以用onehot+Dense(1)代替embedding，但是参数更多，更慢

In [ ]:
class MySumLayer(Layer):
    def __init__(self, axis, **kwargs):
        self.supports_masking = True
        self.axis = axis
        super(MySumLayer, self).__init__(**kwargs)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):

        if mask is not None:
            # mask (batch, time)
            mask = K.cast(mask, K.floatx())
            if K.ndim(x)!=K.ndim(mask):
                mask = K.repeat(mask, x.shape[-1])
                mask = tf.transpose(mask, [0,2,1])
            x = x * mask
            if K.ndim(x)==2:
                x = K.expand_dims(x)
            return K.sum(x, axis=self.axis)
        else:
            if K.ndim(x)==2:
                x = K.expand_dims(x)
            return K.sum(x, axis=self.axis)

    def compute_output_shape(self, input_shape):
        output_shape = []
        for i in range(len(input_shape)):
            if i!=self.axis:
                output_shape.append(input_shape[i])
        if len(output_shape)==1:
            output_shape.append(1)
        return tuple(output_shape)

class MyFlatten(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(MyFlatten, self).__init__(**kwargs)

    def compute_mask(self, inputs, mask=None):
        if mask==None:
            return mask
        return K.batch_flatten(mask)

    def call(self, inputs, mask=None):
        return K.batch_flatten(inputs)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], np.prod(input_shape[1:]))

from sklearn import preprocessing
def simple_nn(train_df,train_y,test_df,val,y2):

    nums = list(train_df.nunique())
    le = preprocessing.LabelEncoder()
    for c in train_df.columns:
        le.fit(train_df[c])
        train_df[c] = le.transform(train_df[c])
        test_df[c] = le.transform(test_df[c])
    
    X = train_df.values
    y = train_y.values

    def slices(x):
        t = []
        for i in range(x.shape[1]):
            ti = K.expand_dims(x[:,i],1)
            t.append(ti)
        return t
            
    def embed(x):
        t = []
        for i in x:
            tx = Dense(1,use_bias=False)(i)
            t.append(tx)
        t = Concatenate(axis=1)(t)
        return t
        
    def catembed0(x):
        t = []
        for i in range(len(x)):
            tx = Embedding(nums[i],1,input_length=1)(x[i])
            t.append(tx)
        t = Add()(t)    
        return Flatten()(t)
        
    def catembed(x):#每个特征最多16个取值，压缩为二维向量
        t = []
        for i in range(len(x)):
            tx = Embedding(nums[i],2,input_length=1)(x[i])
            t.append(tx)
        t = Concatenate(axis=1)(t)
        return t
    
    def get_deepfm():
        xin = Input(shape=(X.shape[1],))
        
        #把特征分成一列列各做embeding
        x0 = Lambda(slices)(xin)
        #一阶
        fd = catembed0(x0)
        
        #二阶
        x0e = catembed(x0)
        x0s = MySumLayer(axis=1)(x0e)
        x02 = Multiply()([x0e, x0e])
        x02s = MySumLayer(axis=1)(x02)
        x0s2 = Multiply()([x0s, x0s])
        sd = Subtract()([x0s2,x02s])
        sd = Lambda(lambda x:x*0.5)(sd)
        sd = MySumLayer(axis=1)(sd)
        
        #MLP
        x1 = MyFlatten()(x0e)   
        x1 = Dense(64,init='he_normal')(x1)
        x1 = PReLU()(x1)
        x1 = BatchNormalization()(x1)
        x1 = Dropout(0.5)(x1)
        x1 = Dense(64,init='he_normal')(x1)
        x1 = PReLU()(x1)
        x1 = BatchNormalization()(x1)
        x1 = Dropout(0.5)(x1)
        x1 = Dense(64,init='he_normal')(x1)
        x1 = PReLU()(x1)
        x1 = BatchNormalization()(x1)
        x1 = Dropout(0.5)(x1)
        x1 = Dense(1, activation='relu')(x1)
    
        x1 = Concatenate(axis=1)([fd, sd,x1])
        y = Dense(1, activation='sigmoid')(x1)
        
        model = Model(inputs=xin,outputs=y)
        return model

    from sklearn.model_selection import KFold
    N = 10
    skf = KFold(n_splits=N,shuffle=False,random_state=None)
    
    xx_cv = []
    xx_pre = []
    ii = 0
    
    for train_in,test_in in skf.split(X,y):
        X_train,X_test,y_train,y_test = X[train_in],X[test_in],y[train_in],y[test_in]
        
        model = get_deepfm()
        model.compile(loss='binary_crossentropy',
                      optimizer = 'nadam',
                      metrics=[auc]
                      )
        if ii==0:            
            print (model.summary())
        
        filepath = fp0+'xgb_deepfm_param'+str(ii)+'.h5'
        bests = 0
        beste = 0
        allres = []
        allval = []
        
        checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
        
        model.fit(X_train,y_train,epochs=2,batch_size=512,
                    validation_data=[X_test,y_test],
                    #validation_data=[test_df,y2],
                    shuffle=True,verbose=2,callbacks = [EarlyStopping(monitor='val_auc',patience=5,mode='max'),checkpoint])
        
        model.load_weights(filepath)
        res1 = model.predict(test_df,batch_size=1000)
        if val:
            s1 = metrics.roc_auc_score(y2, res1)
            print (s1)              
        xx_pre.append(res1)  
        xx_cv.append(bests)
        ii = ii + 1
    
    if val:
        test_y2 = xx_pre[0]
    else:
        test_y2 = 0
        for i in xx_pre:
            test_y2 = test_y2+i
        test_y2 = test_y2/N
    return test_y2